# ✉️ Messages
  <img src="./assets/LC_Messages.png" width="500">

Messages are the fundamental unit of context for models in LangChain. They represent the input and output of models, carrying both the content and metadata needed to represent the state of a conversation when interacting with an LLM.

## Human👨‍💻 and AI 🤖 Messages

In [1]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage

agent = create_agent(
    model="openai:gpt-5-nano", 
    system_prompt="You are a full-stack comedian"
)

In [2]:
human_msg = HumanMessage("Hello, how are you?")

result = agent.invoke({"messages": [human_msg]})

In [3]:
print(result["messages"][-1].content)

Doing great, thanks! I’m fueled by coffee and clean code—front-end is beaming, back-end humming, and my cache is freshly warmed. How about you? Any bugs you want me to squash or features you’re chasing today?


In [4]:
print(type(result["messages"][-1]))

<class 'langchain_core.messages.ai.AIMessage'>


In [5]:
for msg in result["messages"]:
    print(f"{msg.type}: {msg.content}\n")

human: Hello, how are you?

ai: Doing great, thanks! I’m fueled by coffee and clean code—front-end is beaming, back-end humming, and my cache is freshly warmed. How about you? Any bugs you want me to squash or features you’re chasing today?



### Altenative formats
#### Strings
There are situations where LangChain can infer the role from the context, and a simple string is enough to create a message. 

In [ ]:
agent = create_agent(
    model="openai:gpt-5-nano",
    system_prompt="You are a terse sports poet.",  # This is a SystemMessage under the hood
)

In [ ]:
result = agent.invoke({"messages": "Tell me about baseball"})   # This is a HumanMessage under the hood
print(result["messages"][-1].content)

Baseball: a diamond carved in dirt and sun.
Pitcher vs. batter, the old duel, glove ready, bat awake.
Three strikes, four balls, the count grows heavy.
Crack—the bat; leather sings; the field holds its breath.
Grounders scoot, a sprint to first, a slide in dust.
Outs stack, innings flip, the scoreboard breathes.
Nine on the field, a touch of chaos and grace.
Bases whisper runs—steal, tag, safe—crowds rise.
Season turns, legends bloom from dirt and air.
Baseball: memory kept in chalk, sweat, and seam.


#### Dictionaries

In [8]:
result = agent.invoke(
    {"messages": {"role": "user", "content": "Write a haiku about sprinters"}}
)
print(result["messages"][-1].content)

Lightning in the lane
Muscle and breath split the air
Finish lines hush, go


There are multiple roles:
```python
messages = [
    {"role": "system", "content": "You are a sports poetry expert who completes haikus that have been started"},
    {"role": "user", "content": "Write a haiku about sprinters"},
    {"role": "assistant", "content": "Feet don't fail me..."}
]
```

## Output Format
### messages
Let's create a tool so agent will create some tool messages. 

In [9]:
from langchain_core.tools import tool

@tool
def check_haiku_lines(text: str):
    """Check if the given haiku text has exactly 3 lines.

    Returns None if it's correct, otherwise an error message.
    """
    # Split the text into lines, ignoring leading/trailing spaces
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    print(f"checking haiku, it has {len(lines)} lines:\n {text}")

    if len(lines) != 3:
        return f"Incorrect! This haiku has {len(lines)} lines. A haiku must have exactly 3 lines."
    return "Correct, this haiku has 3 lines."

In [10]:
agent = create_agent(
    model="openai:gpt-5",
    tools=[check_haiku_lines],
    system_prompt="You are a sports poet who only writes Haiku. You always check your work.",
)

In [11]:
result = agent.invoke({"messages": "Please write me a poem"})

checking haiku, it has 3 lines:
 Whistle splits cool dusk
Leather arcs, the net whispers
Streetlights crown the game


In [12]:
result["messages"][-1].content

'Whistle splits cool dusk\nLeather arcs, the net whispers\nStreetlights crown the game'

In [13]:
print(len(result["messages"]))

4


In [14]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()

================================ Human Message =================================

Please write me a poem
================================== Ai Message ==================================
Tool Calls:
  check_haiku_lines (call_nJf9uhb7A4OrStGXAoPXARBJ)
 Call ID: call_nJf9uhb7A4OrStGXAoPXARBJ
  Args:
    text: Whistle splits cool dusk
Leather arcs, the net whispers
Streetlights crown the game
================================= Tool Message =================================
Name: check_haiku_lines

Correct, this haiku has 3 lines.
================================== Ai Message ==================================

Whistle splits cool dusk
Leather arcs, the net whispers
Streetlights crown the game


### Other useful information
Above, the print messages have just been selecting pieces of the information stored in the messages list. Let's dig into all the information that is available!

In [15]:
result

{'messages': [HumanMessage(content='Please write me a poem', additional_kwargs={}, response_metadata={}, id='09a23f47-3695-4c49-958d-7920aa25e179'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 747, 'prompt_tokens': 170, 'total_tokens': 917, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 704, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-COQd90XIcrTD0Y6qU3qW1LBK0C4oR', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--fd44e060-62b7-48df-9a21-80d353c178a1-0', tool_calls=[{'name': 'check_haiku_lines', 'args': {'text': 'Whistle splits cool dusk\nLeather arcs, the net whispers\nStreetlights crown the game'}, 'id': 'call_nJf9uhb7A4OrStGXAoPXARBJ', 'type': 'tool_call'}],

You can select just the last message, and you can see where the final message is coming from.

In [16]:
result["messages"][-1]

AIMessage(content='Whistle splits cool dusk\nLeather arcs, the net whispers\nStreetlights crown the game', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 231, 'total_tokens': 252, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-COQdJoeLoCXGruWwXs6hLJFakmCpS', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--6a71bb7f-3880-4377-8aae-ab3d8896a4de-0', usage_metadata={'input_tokens': 231, 'output_tokens': 21, 'total_tokens': 252, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [17]:
result["messages"][-1].usage_metadata

{'input_tokens': 231,
 'output_tokens': 21,
 'total_tokens': 252,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 0}}

In [18]:
result["messages"][-1].response_metadata

{'token_usage': {'completion_tokens': 21,
  'prompt_tokens': 231,
  'total_tokens': 252,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
 'model_provider': 'openai',
 'model_name': 'gpt-5-2025-08-07',
 'system_fingerprint': None,
 'id': 'chatcmpl-COQdJoeLoCXGruWwXs6hLJFakmCpS',
 'service_tier': 'default',
 'finish_reason': 'stop',
 'logprobs': None}

### Try it on your own!
Change the system prompt, use the `pretty_printer` to print some messages or dig through `results` on your own. Notice the Human, AI and Tool messages and some of their associated metadata. Notice how the final results provide a complete history of the agents activity!

In [21]:
agent = create_agent(
    model="openai:gpt-5",
    tools=[check_haiku_lines],
    system_prompt="Your SYSTEM prompt here",
)

In [ ]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()